<a href="https://colab.research.google.com/github/leonardoub/SCRIPT_PALERMO/blob/master/prova_hyperparameter_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install -U keras-tuner

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [0]:
!rm -r my_dir

#Load data

In [259]:
#load data from Drive
from google.colab import drive
drive.mount('/gdrive')
#%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
train_dataset_path = '/gdrive/My Drive/AIM_PA/database_training2.csv'
test_dataset_path = '/gdrive/My Drive/AIM_PA/database_nostro_without_nan.csv'

In [0]:
df_train = pd.read_csv(train_dataset_path)
df_test = pd.read_csv(test_dataset_path)

In [0]:
df_train.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_test.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_train.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)
df_test.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)

In [0]:
train_data = df_train.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
test_data = df_test.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
train_labels = df_train.Histology

In [0]:
test_labels = df_test.Histology

##Z score dei dati

In [0]:
mean = train_data.mean(axis=0)
train_data_stand = train_data - mean
std = train_data.std(axis=0)
train_data_stand /= std

In [0]:
test_data_stand = test_data - mean
test_data_stand /= std

##Vettorizzare i label

In [0]:
word_index={'adenocarcinoma':0, 'large cell':1, 'squamous cell carcinoma':2}

In [0]:
train_labels_dec = [word_index[label] for label in train_labels]

In [0]:
test_labels_dec = [word_index[label] for label in test_labels]

In [0]:
from keras.utils.np_utils import to_categorical

In [0]:
one_hot_train_labels = to_categorical(train_labels_dec)
one_hot_test_labels = to_categorical(test_labels_dec)

#PCA

In [0]:
from sklearn.decomposition import PCA

In [0]:
pca = PCA(n_components=0.9, svd_solver='full')

In [278]:
pca.fit(train_data_stand)

PCA(copy=True, iterated_power='auto', n_components=0.9, random_state=None,
    svd_solver='full', tol=0.0, whiten=False)

In [0]:
train_data_stand_pca = pca.transform(train_data_stand)
test_data_stand_pca = pca.transform(test_data_stand)

In [280]:
train_data_stand_pca.shape

(131, 9)

#Building Network

In [0]:
from tensorflow import keras

In [0]:
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence

In [0]:
from tensorflow.keras import layers

In [0]:
from tensorflow.keras import regularizers

In [0]:
from tensorflow.keras.optimizers import SGD

In [0]:
from kerastuner.tuners import RandomSearch

In [0]:
def build_model(hp):
  model = keras.models.Sequential()
  activation = hp.Choice('activation', ['softmax', 'softplus', 'softsign',
                              'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'])
  
  weight_init = hp.Choice('weight_init', 
                            ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'])
  
  drop_rate = hp.Choice('drop_rate', [0.0, 0.1, 0.2, 0.3,
                              0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
  
  model.add(layers.Dense(units=(hp.Int('units', min_value=3, max_value=8, step=1)), 
                         activation=activation, kernel_initializer=weight_init, input_shape=(9,)))
  
  model.add(layers.Dropout(rate=drop_rate))

  model.add(layers.Dense(3, activation='softmax'))
  #sgd_1 = SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True)
 
  #optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])

  lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
  momentum = hp.Choice('momentum', [0.0, 0.2, 0.4, 0.6, 0.8, 0.9])
  
  model.compile(optimizer=keras.optimizers.SGD(lr, momentum=momentum), 
                loss='categorical_crossentropy', metrics=['accuracy'])
  return model

#Stratified k-fold

This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.

In [0]:
from sklearn.model_selection import StratifiedKFold

In [289]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
skf.get_n_splits(train_data_stand_pca, train_labels_dec)

3

In [290]:
for train_index, test_index in skf.split(train_data_stand_pca, train_labels_dec):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [  0   1   4   5   8   9  11  12  14  15  16  17  19  20  22  23  24  25
  27  29  30  33  34  36  37  38  39  40  41  42  44  45  46  48  51  52
  53  56  57  58  59  60  62  63  65  66  67  69  72  76  77  78  79  80
  81  83  84  85  87  88  89  90  92  96  97  98 100 101 102 103 104 105
 107 109 110 111 113 115 117 120 121 122 124 125 127 128] TEST: [  2   3   6   7  10  13  18  21  26  28  31  32  35  43  47  49  50  54
  55  61  64  68  70  71  73  74  75  82  86  91  93  94  95  99 106 108
 112 114 116 118 119 123 126 129 130]
TRAIN: [  2   3   5   6   7   8   9  10  11  12  13  18  20  21  25  26  27  28
  29  30  31  32  34  35  36  38  39  43  44  45  46  47  48  49  50  53
  54  55  57  58  61  63  64  65  66  68  70  71  73  74  75  76  78  82
  84  85  86  87  90  91  92  93  94  95  96  99 100 101 102 105 106 108
 109 111 112 114 115 116 118 119 122 123 124 125 126 127 129 130] TEST: [  0   1   4  14  15  16  17  19  22  23  24  33  37  40  41  42  51  52
  56  59 

In [291]:
train_labels_dec[125]

2

In [0]:
from keras.utils import to_categorical


In [0]:
#for train_index, val_index in skf.split(train_data_stand_pca, train_labels_dec):
 
#  partial_train_data = np.array([train_data_stand_pca[i] for i in train_index])
#  partial_train_targets = np.array([train_labels_dec[i] for i in train_index])

#  val_data = np.array([train_data_stand_pca[i] for i in val_index])
#  val_targets = np.array([train_labels_dec[i] for i in val_index])

#  one_hot_partial_train_targets = to_categorical(partial_train_targets)
#  one_hot_val_targets = to_categorical(val_targets)

#  model = build_model()
#  model.fit(partial_train_data, one_hot_partial_train_targets, epochs = num_epochs, batch_size=1)

#  val_loss, val_accuracy = model.evaluate(val_data, one_hot_val_targets)
#  all_scores.append(val_accuracy)
#I parametri per la valutazione vengono calcolati una volta per ogni k-fold, per ogni set di validazione, quindi k volte

C'è un problema: keras.utils.to_categorical produces a one-hot encoded class vector, i.e. the multilabel-indicator mentioned in the error message. StratifiedKFold is not designed to work with such input; i.e. your y must be a 1-D array of your class labels.
Essentially, what you have to do is simply to invert the order of the operations: split first (using your intial y_train), and convert to_categorical afterwards.

In [294]:
num_epochs = 50
all_acc_histories = []
all_loss_histories = []
all_val_acc_histories = []
all_val_loss_histories = []

for train_index, val_index in skf.split(train_data_stand_pca, train_labels_dec):
 
  partial_train_data = np.array([train_data_stand_pca[i] for i in train_index])
  partial_train_targets = np.array([train_labels_dec[i] for i in train_index])
  
  val_data = np.array([train_data_stand_pca[i] for i in val_index])
  val_targets = np.array([train_labels_dec[i] for i in val_index])

  one_hot_partial_train_targets = to_categorical(partial_train_targets)
  one_hot_val_targets = to_categorical(val_targets)

 
  tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=20, 
                       executions_per_trial=5, directory='/content/my_dir', project_name='helloworld')
  
  tuner.search_space_summary()

  tuner.search(partial_train_data, one_hot_partial_train_targets, validation_data=(val_data, one_hot_val_targets), 
                      epochs=num_epochs, batch_size=10)
  

#  acc_history = history.history['acc']
#  all_acc_histories.append(acc_history)

#  loss_history = history.history['loss']
#  all_loss_histories.append(loss_history)

#  acc_val_history = history.history['val_acc']
#  all_val_acc_histories.append(acc_val_history)

#  loss_val_history = history.history['val_loss']
#  all_val_loss_histories.append(loss_val_history)
  

#I parametri per la valutazione vengono calcolati per ogni epoca, quindi num_epochs volte. 
#Il tutto viene ripetuto un numero di volte pari a n_splits.
#Si ottiene una lista con n_splits elementi ciascuno dei quali è una lista lunga num_epochs,
#ogni elemento può essere uno fra questi: dict_keys(['val_loss', 'val_acc', 'loss', 'acc']) 

Train on 86 samples, validate on 45 samples
Epoch 1/50
86/86 [==============================] - 0s 5ms/sample - loss: 1.6687 - accuracy: 0.3140 - val_loss: 1.4419 - val_accuracy: 0.2667
Epoch 2/50
86/86 [==============================] - 0s 463us/sample - loss: 1.6234 - accuracy: 0.3140 - val_loss: 1.4320 - val_accuracy: 0.2667
Epoch 3/50
86/86 [==============================] - 0s 585us/sample - loss: 1.4648 - accuracy: 0.2907 - val_loss: 1.4269 - val_accuracy: 0.2889
Epoch 4/50
86/86 [==============================] - 0s 531us/sample - loss: 1.6276 - accuracy: 0.3488 - val_loss: 1.4188 - val_accuracy: 0.2889
Epoch 5/50
86/86 [==============================] - 0s 493us/sample - loss: 1.5055 - accuracy: 0.3023 - val_loss: 1.4128 - val_accuracy: 0.2889
Epoch 6/50
86/86 [==============================] - 0s 486us/sample - loss: 1.7753 - accuracy: 0.2209 - val_loss: 1.4046 - val_accuracy: 0.2889
Epoch 7/50
86/86 [==============================] - 0s 455us/sample - loss: 1.4956 - accuracy:

Train on 86 samples, validate on 45 samples
Epoch 1/50
86/86 [==============================] - 0s 4ms/sample - loss: 1.3903 - accuracy: 0.2907 - val_loss: 1.2461 - val_accuracy: 0.2444
Epoch 2/50
86/86 [==============================] - 0s 413us/sample - loss: 1.3274 - accuracy: 0.3488 - val_loss: 1.2448 - val_accuracy: 0.2444
Epoch 3/50
86/86 [==============================] - 0s 425us/sample - loss: 1.2856 - accuracy: 0.3256 - val_loss: 1.2434 - val_accuracy: 0.2222
Epoch 4/50
86/86 [==============================] - 0s 410us/sample - loss: 1.2912 - accuracy: 0.2674 - val_loss: 1.2417 - val_accuracy: 0.2222
Epoch 5/50
86/86 [==============================] - 0s 446us/sample - loss: 1.4198 - accuracy: 0.2442 - val_loss: 1.2401 - val_accuracy: 0.2222
Epoch 6/50
86/86 [==============================] - 0s 415us/sample - loss: 1.3119 - accuracy: 0.2907 - val_loss: 1.2383 - val_accuracy: 0.2222
Epoch 7/50
86/86 [==============================] - 0s 503us/sample - loss: 1.3261 - accuracy:

Train on 86 samples, validate on 45 samples
Epoch 1/50
86/86 [==============================] - 0s 4ms/sample - loss: 2.0784 - accuracy: 0.5349 - val_loss: 2.0394 - val_accuracy: 0.4444
Epoch 2/50
86/86 [==============================] - 0s 549us/sample - loss: 2.1737 - accuracy: 0.5465 - val_loss: 1.9805 - val_accuracy: 0.5333
Epoch 3/50
86/86 [==============================] - 0s 459us/sample - loss: 1.8411 - accuracy: 0.4884 - val_loss: 1.9007 - val_accuracy: 0.5111
Epoch 4/50
86/86 [==============================] - 0s 378us/sample - loss: 1.7411 - accuracy: 0.4302 - val_loss: 1.8185 - val_accuracy: 0.5111
Epoch 5/50
86/86 [==============================] - 0s 415us/sample - loss: 1.7731 - accuracy: 0.3953 - val_loss: 1.6149 - val_accuracy: 0.4667
Epoch 6/50
86/86 [==============================] - 0s 449us/sample - loss: 1.5421 - accuracy: 0.5000 - val_loss: 1.5475 - val_accuracy: 0.4444
Epoch 7/50
86/86 [==============================] - 0s 396us/sample - loss: 1.2801 - accuracy:

Train on 86 samples, validate on 45 samples
Epoch 1/50
86/86 [==============================] - 0s 4ms/sample - loss: 1.0729 - accuracy: 0.3721 - val_loss: 1.0665 - val_accuracy: 0.4667
Epoch 2/50
86/86 [==============================] - 0s 461us/sample - loss: 1.2566 - accuracy: 0.3023 - val_loss: 1.0651 - val_accuracy: 0.4667
Epoch 3/50
86/86 [==============================] - 0s 419us/sample - loss: 1.2181 - accuracy: 0.3837 - val_loss: 1.0634 - val_accuracy: 0.4667
Epoch 4/50
86/86 [==============================] - 0s 429us/sample - loss: 1.1453 - accuracy: 0.4767 - val_loss: 1.0617 - val_accuracy: 0.4667
Epoch 5/50
86/86 [==============================] - 0s 434us/sample - loss: 1.2396 - accuracy: 0.3605 - val_loss: 1.0599 - val_accuracy: 0.4667
Epoch 6/50
86/86 [==============================] - 0s 446us/sample - loss: 1.0686 - accuracy: 0.4535 - val_loss: 1.0588 - val_accuracy: 0.4667
Epoch 7/50
86/86 [==============================] - 0s 419us/sample - loss: 1.1595 - accuracy:

Train on 86 samples, validate on 45 samples
Epoch 1/50
86/86 [==============================] - 0s 4ms/sample - loss: 1.9326 - accuracy: 0.3721 - val_loss: 1.1611 - val_accuracy: 0.3333
Epoch 2/50
86/86 [==============================] - 0s 385us/sample - loss: 2.0022 - accuracy: 0.3721 - val_loss: 1.1573 - val_accuracy: 0.3111
Epoch 3/50
86/86 [==============================] - 0s 412us/sample - loss: 1.9693 - accuracy: 0.3837 - val_loss: 1.1553 - val_accuracy: 0.3111
Epoch 4/50
86/86 [==============================] - 0s 422us/sample - loss: 1.4646 - accuracy: 0.4419 - val_loss: 1.1538 - val_accuracy: 0.3111
Epoch 5/50
86/86 [==============================] - 0s 409us/sample - loss: 1.9390 - accuracy: 0.4767 - val_loss: 1.1517 - val_accuracy: 0.3111
Epoch 6/50
86/86 [==============================] - 0s 455us/sample - loss: 1.7919 - accuracy: 0.3953 - val_loss: 1.1498 - val_accuracy: 0.3111
Epoch 7/50
86/86 [==============================] - 0s 408us/sample - loss: 1.6737 - accuracy:

...

Train on 86 samples, validate on 45 samples
Epoch 1/50
86/86 [==============================] - 0s 4ms/sample - loss: 1.2535 - accuracy: 0.1395 - val_loss: 1.1633 - val_accuracy: 0.1333
Epoch 2/50
86/86 [==============================] - 0s 539us/sample - loss: 1.1637 - accuracy: 0.2558 - val_loss: 1.1081 - val_accuracy: 0.3111
Epoch 3/50
86/86 [==============================] - 0s 494us/sample - loss: 1.0786 - accuracy: 0.3837 - val_loss: 1.0682 - val_accuracy: 0.5556
Epoch 4/50
86/86 [==============================] - 0s 369us/sample - loss: 1.0233 - accuracy: 0.5000 - val_loss: 1.0461 - val_accuracy: 0.4667
Epoch 5/50
86/86 [==============================] - 0s 437us/sample - loss: 1.0312 - accuracy: 0.4535 - val_loss: 1.0306 - val_accuracy: 0.5111
Epoch 6/50
86/86 [==============================] - 0s 488us/sample - loss: 0.9749 - accuracy: 0.5465 - val_loss: 1.0222 - val_accuracy: 0.4667
Epoch 7/50
86/86 [==============================] - 0s 387us/sample - loss: 0.9624 - accuracy:

Train on 86 samples, validate on 45 samples
Epoch 1/50
86/86 [==============================] - 0s 4ms/sample - loss: 1.9552 - accuracy: 0.5349 - val_loss: 1.3184 - val_accuracy: 0.4889
Epoch 2/50
86/86 [==============================] - 0s 528us/sample - loss: 1.4426 - accuracy: 0.5000 - val_loss: 1.2340 - val_accuracy: 0.5333
Epoch 3/50
86/86 [==============================] - 0s 467us/sample - loss: 1.2185 - accuracy: 0.5233 - val_loss: 1.1490 - val_accuracy: 0.4889
Epoch 4/50
86/86 [==============================] - 0s 387us/sample - loss: 1.2003 - accuracy: 0.4884 - val_loss: 1.0782 - val_accuracy: 0.4667
Epoch 5/50
86/86 [==============================] - 0s 413us/sample - loss: 1.0444 - accuracy: 0.4535 - val_loss: 1.0219 - val_accuracy: 0.4667
Epoch 6/50
86/86 [==============================] - 0s 416us/sample - loss: 1.1054 - accuracy: 0.5349 - val_loss: 0.9987 - val_accuracy: 0.4667
Epoch 7/50
86/86 [==============================] - 0s 419us/sample - loss: 1.0381 - accuracy:

INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Reloading Oracle from /content/my_dir/helloworld/oracle.json
INFO:tensorflow:Reloading Tuner from /content/my_dir/helloworld/tuner0.json


INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Reloading Oracle from /content/my_dir/helloworld/oracle.json
INFO:tensorflow:Reloading Tuner from /content/my_dir/helloworld/tuner0.json


INFO:tensorflow:Oracle triggered exit


In [295]:
tuner.search_space_summary()


In [296]:
tuner.results_summary()

In [297]:
tuner.get_best_hyperparameters()

In [298]:
tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

{'activation': 'linear',
 'drop_rate': 0.3,
 'learning_rate': 0.01,
 'momentum': 0.8,
 'units': 7,
 'weight_init': 'lecun_uniform'}

In [0]:
best_model = tuner.get_best_models()[0]

In [0]:
#history_dict = history.history

In [0]:
#history_dict.keys()

In [0]:
#len(all_acc_histories[2])

In [0]:
#average_acc_history = [np.mean([x[i] for x in all_acc_histories]) for i in range(num_epochs)]
##media per epoca degli score ottenuti per tutte le k-fold
##per ogni k-fold di fanno num_epoch epoche, la media viene fatta prendendo gli score di tutti i k-fold relativi ad una data epoca,
##e si fa questo per tutte le epoche
#average_loss_history = [np.mean([x[i] for x in all_loss_histories]) for i in range(num_epochs)]
#average_val_acc_history = [np.mean([x[i] for x in all_val_acc_histories]) for i in range(num_epochs)]
#average_val_loss_history = [np.mean([x[i] for x in all_val_loss_histories]) for i in range(num_epochs)]


In [0]:
#len(average_val_acc_history)

##Plotting training and validation loss

In [0]:
import matplotlib.pyplot as plt

In [0]:
epochs = range(1, num_epochs+1)

In [306]:
plt.plot(epochs, average_loss_history, 'bo', label='training loss')
plt.plot(epochs, average_val_loss_history, 'b', label='validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

NameError: ignored

##Plotting train and validation accuracy

In [0]:
plt.plot(epochs, average_acc_history, 'bo', label='Training accuracy')
plt.plot(epochs, average_val_acc_history, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend() 

In [0]:
!pwd

In [0]:
!pip list